In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier

from sklearn.ensemble import StackingClassifier
from sklearn.decomposition import FactorAnalysis

from sklearn.preprocessing import scale
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import StackingClassifier

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
l_train = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/sample_submission.csv')
fin_cols =l_train.columns.to_list()

In [ ]:
f_train = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/train.csv')
f_test = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/test.csv')
#l_train = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/sample_submission.csv')

In [ ]:
encoder= LabelEncoder().fit(f_train['target'])
f_train['target'] = encoder.transform(f_train['target'])
f_train = f_train.drop(['id'], axis = 1)

In [ ]:
X = f_train.drop(['target'], axis = 1)
Y = f_train['target']


X_train, X_test, Y_train, Y_test = train_test_split(X, 
                                                 Y, 
                                                 test_size=0.2)

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!nvidia-smi

In [ ]:
%%time
#XGBoost + GridSearchCV  model

#cv_split = ShuffleSplit(n_splits = 10, test_size = .3, train_size = .6, random_state = 0 )

#grid_n_estimator = [30, 50, 80]
#grid_seed = [89]
#grid_learn = [.0007, .001, .003]

#XGB = XGBClassifier(num_class=4, 
#                    objective='multi:softprob', 
#                    tree_method = 'gpu_hist', 
#                    eval_metric = 'auc')
XGB = XGBClassifier(num_class=4, 
                    objective='multi:softprob', 
                    seed = 89,
                    n_estimators = 2000,
                    max_depth = 3,
                    learning_rate = .3)
#XGB = GridSearchCV(XGBClassifier(), param_grid= {'learning_rate': grid_learn, 
#                                                   'max_depth': [3,6], 
#                                                   'n_estimators': grid_n_estimator, 
#                                                   'seed': grid_seed}, 
#                     scoring = 'roc_auc', cv = cv_split)


#XGB.fit(X, Y)
#print('Best Parameters: ', XGB.best_params_)
#Y_pred_train = XGB.predict(X)
#print(classification_report(Y, Y_pred_train))

In [ ]:
%%time
#CatBoost + GridSearchCV  model

#cv_split = ShuffleSplit(n_splits = 10, test_size = .3, train_size = .6, random_state = 0 )

#grid_n_estimator = [50, 100, 500]
#grid_learn = [.001, .01, .1]

#Cat = CatBoostClassifier(iterations = 1, loss_function='MultiClass', task_type="GPU")

Cat = CatBoostClassifier(loss_function = 'MultiClass',
                         task_type="GPU",
                         n_estimators = 1500,
                         depth = 4,
                         learning_rate = 0.2)

#Cat = GridSearchCV(CatBoostClassifier(), param_grid= {'learning_rate': grid_learn,
#                                                        'depth': [4,5,6],
#                                                        'n_estimators': grid_n_estimator},
#                     scoring = 'roc_auc', cv = cv_split)

#Cat.fit(X_train, Y_train, verbose=False)
#print('Best Parameters: ', Cat.best_params_)
#Y_pred_train = Cat.predict(X_train)
#print( classification_report(Y_train, Y_pred_train))

#Y_pred = Cat.predict(X_test)

#print("Stacking model score: %.3f" % Cat.score(X_test, Y_test))

In [ ]:
%%time
cv_split = StratifiedKFold(n_splits = 3, shuffle = True)

estimators = [('XGB', XGB),
              ('Cat', Cat)] 

stacked = StackingClassifier(estimators = estimators, 
                             final_estimator = LogisticRegression(),
                             verbose = 4, 
                             cv = cv_split)

stacked.fit(X_train, Y_train)

Y_pred_train = stacked.predict(X_train)
print( classification_report(Y_train, Y_pred_train))

Y_pred = stacked.predict(X_test)

print("Stacking model score: %.3f" % stacked.score(X_test, Y_test))

In [ ]:
result = stacked.predict_proba(f_test.drop(['id'], axis =1))
result = pd.DataFrame(result, columns = fin_cols[1:])
result = result.join(f_test['id'],on=None, how='left')
result = result[['id', 'Class_1', 'Class_2', 'Class_3', 'Class_4']]
result

In [ ]:
result.to_csv("Stacked_result.csv", index=False)